# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

## 1:

In [9]:
# Tratando valores ausentes em 'tempo_emprego':
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].median())

# Criando variável logarítmica da renda:
df['log_renda'] = np.log(df['renda'])

# Definindo a fórmula para o modelo incluindo todas as variáveis:
# Usando Patsy para criar as variáveis dummy e excluir a categoria de referência:
formula = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

# Construindo o design matrix e a resposta usando Patsy:
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

# Ajustando o modelo de regressão linear múltipla:
model = sm.OLS(y, X).fit()

# Resumo do modelo:
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sun, 28 Jul 2024   Prob (F-statistic):               0.00
Time:                        10:50:26   Log-Likelihood:                -16213.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14975   BIC:                         3.267e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

### Análise:


Em termos práticos,  temos variáveis como tempo_emprego, posse_de_veiculo, posse_de_imovel, além das variáveis de tipo de renda (tipo_renda[T.Bolsista], tipo_renda[T.Empresário],tipo_renda[T.Pensionista] tipo_renda[T.Servidor público]), por exemplo, que nos ajudam a explicar e a prever o aumento de renda.

_______________

## 2:

In [14]:
# Preenchendo valores ausentes:
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].median())

# Criando a variável logarítmica da renda:
df['log_renda'] = np.log(df['renda'])

# Definindo a fórmula inicial para o modelo:
formula_inicial = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

# Construindo a design matrix e a resposta usando Patsy:
y, X = patsy.dmatrices(formula_inicial, data=df, return_type='dataframe')

# Ajustando o modelo de regressão linear múltipla:
model_inicial = sm.OLS(y, X).fit()

# Resumo do modelo inicial:
print("Resumo do Modelo Inicial:")
print(model_inicial.summary())

# Atualizando a fórmula para excluir as variáveis 'educacao' e 'tipo_residencia':
formula_reduzida = formula_inicial.replace('educacao + ', '')
formula_reduzida = formula_reduzida.replace(' + educacao', '')
formula_reduzida = formula_reduzida.replace('tipo_residencia + ', '')
formula_reduzida = formula_reduzida.replace(' + tipo_residencia', '')

# Construindo a design matrix e a resposta usando Patsy:
y_reduzido, X_reduzido = patsy.dmatrices(formula_reduzida, data=df, return_type='dataframe')

# Ajustando o modelo reduzido (sem 'educacao' e 'tipo_residencia'):
model_reduzido = sm.OLS(y_reduzido, X_reduzido).fit()

# Resumo do modelo reduzido:
print("\nResumo do Modelo com 'educacao' e 'tipo_residencia' removidas:")
print(model_reduzido.summary())


Resumo do Modelo Inicial:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sun, 28 Jul 2024   Prob (F-statistic):               0.00
Time:                        10:50:29   Log-Likelihood:                -16213.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14975   BIC:                         3.267e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

### Análise:

Aqui nós temos uma pequena diminuição do R-quadrado, com a remoção das variáveis educacao e tipo_residencia. O modelo fica, assim, ajustado.

_______________

## 3:

In [39]:
# Tratando valores ausentes em 'tempo_emprego':
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].median())

# Criando a variável logarítmica da renda:
df['log_renda'] = np.log(df['renda'])

# Definindo a fórmula inicial para o modelo:
formula_inicial = (
    'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + '
    'tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + '
    'qt_pessoas_residencia'
)

# Construindo a design matrix e a resposta usando Patsy:
y, X = patsy.dmatrices(formula_inicial, data=df, return_type='dataframe')

# Ajustando o modelo de regressão linear múltipla:
model_inicial = sm.OLS(y, X).fit()

# Resumo do modelo inicial:
print("Resumo do Modelo Inicial:")
print(model_inicial.summary())

# Atualizando a fórmula para excluir as variáveis 'educacao' e 'tipo_residencia':
formula_reduzida = formula_inicial.replace('educacao + ', '')
formula_reduzida = formula_reduzida.replace(' + educacao', '')
formula_reduzida = formula_reduzida.replace('tipo_residencia + ', '')
formula_reduzida = formula_reduzida.replace(' + tipo_residencia', '')

# Construindo a design matrix e a resposta usando Patsy:
y_reduzido, X_reduzido = patsy.dmatrices(formula_reduzida, data=df, return_type='dataframe')

# Ajustando o modelo reduzido (sem 'educacao' e 'tipo_residencia'):
model_reduzido = sm.OLS(y_reduzido, X_reduzido).fit()

# Resumo do modelo reduzido:
print("\nResumo do Modelo com 'educacao' e 'tipo_residencia' removidas:")
print(model_reduzido.summary())

# Atualizando a fórmula para excluir também as variáveis 'sexo' e 'estado_civil':
formula_final = formula_reduzida.replace('sexo + ', '')
formula_final = formula_final.replace(' + sexo', '')
formula_final = formula_final.replace('estado_civil + ', '')
formula_final = formula_final.replace(' + estado_civil', '')

# Construindo a design matrix e a resposta usando Patsy:
y_final, X_final = patsy.dmatrices(formula_final, data=df, return_type='dataframe')

# Ajustando o modelo final (sem 'sexo', 'estado_civil', 'educacao' e 'tipo_residencia'):
model_final = sm.OLS(y_final, X_final).fit()

# Resumo do modelo final:
print("\nResumo do Modelo com 'sexo', 'estado_civil', 'educacao' e 'tipo_residencia' removidas:")
print(model_final.summary())

# Comparando o modelo inicial e o modelo final:
print("\nComparação entre Modelo Inicial e Modelo Final:")
print("R² do modelo inicial:", model_inicial.rsquared)
print("R² do modelo final:", model_final.rsquared)
print("MSE do modelo inicial:", np.mean(model_inicial.resid**2))
print("MSE do modelo final:", np.mean(model_final.resid**2))


Resumo do Modelo Inicial:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sun, 28 Jul 2024   Prob (F-statistic):               0.00
Time:                        11:13:46   Log-Likelihood:                -16213.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14975   BIC:                         3.267e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

### Análise:

Comparado ao modelo incial, constatamos que o modelo atual atende muito melhor, com apenas as variáveis relevantes para a apuração da renda.